---

<div align=center>

# SETUP

</div>

## Config

In [1]:
version = "1.20.4"

## Install dependencies

In [5]:
%pip install -r requirements.txt

## Import dependencies

In [2]:
import os
import numpy as np
from libs import directory
import urllib

## Global data

In [3]:
datapack_path = os.path.abspath("../datapacks/Bookshelf")

---

<div align=center>

# GENERATORS

</div>